In [ ]:
import intake
import pandas as pd

df = intake.open_csv('./data/bird_migration/{species}.csv').read()

### Adding another layer of data

It is often useful to add another layer of information under a dataset. In this case we might suspect that birds are motivated to migrate because of changing temperature. We will explore that hypothesis by importing data from a global climate model. For this we will use `xarray`.

In [ ]:
import os
import xarray as xr
import hvplot.xarray
import hvplot.pandas

import panel as pn

In [ ]:
data_url = 'http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep/air.day.ltm.nc'

# I downloaded the file locally because I was hitting rate limits.
local_file = './data/air.day.ltm.nc'
if os.path.isfile(local_file):
    data_url = local_file
    
ds = xr.open_dataset(data_url)

The time is in a strange format because it is day of year means for each grid cell. We can convert those to integer day of year like we have for our bird data.

In [ ]:
ds = ds.rename(time='day')
ds['day'] = list(range(1,366))
ds.day

Since we are interested in the temperature near the surface of the earth, we only really need the 1000mbar level. We can select that directly from the `xarray.dataset` and convert the temperature from kelvin to celsius while we are at it.

In [ ]:
air = ds.sel(level=1000, drop=True).air - 273

In [ ]:
air.to_netcdf("data/air.nc")

In [ ]:
air.mean("day").hvplot("lon", "lat")

Since the bird plot and the air plot are both `holoviews` object on the same axes, we can combine 

In [ ]:
air.hvplot("lon", "lat", groupby="day") * \
df.hvplot.points("lon", "lat", groupby="day")

Hmmm. That doesn't look great. Turns out that the birds and the temperature use different conventions for longitude. This is a great time to realize that *all* of these data really belong in a geographic context.

In [ ]:
air.hvplot("lon", "lat", groupby="day", geo=True, cmap="coolwarm", clim=(-40, 40), coastline=True) * \
df.hvplot.points("lon", "lat", groupby=["day", "species"], geo=True, color="black")

[Next Section](./03_geoviews.ipynb#Geoviews)